Gentetic Algorithm 실습
Reference : https://towardsdatascience.com/evolution-of-a-salesman-a-complete-genetic-algorithm-tutorial-for-python-6fe5d2b3ca35
1. Create population
2. Determine fitness
3. Select the mating pool
4. Breed
5. Mutate
6. Repeat

In [21]:
import numpy as np, random, operator, pandas as pd, matplotlib.pyplot as plt,time

In [2]:
#Create City, Fitness Class

class City:
    def __init__(self,x,y):
        self.x = x
        self.y = y
    
    def distance(self,city):
        xDis = abs(self.x-city.x)
        yDis = abs(self.y-city.y)
        distance = np.sqrt((xDis**2)+(yDis**2))
        return distance
    
    def __repr__(self):
        return "("+str(self.x)+","+str(self.y)+")"
    
        

In [3]:
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness= 0.0
    
    def routeDistance(self):
        if self.distance ==0:
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None
                if i + 1 < len(self.route):
                    toCity = self.route[i + 1]
                else:
                    toCity = self.route[0]
                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance
    
    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness

Create Population

In [4]:
#하나의 individual route 생성하는 함수
def createRoute(cityList):
    route = random.sample(cityList,len(cityList))
    return route

In [5]:
#여러 individual route 생성하여 population 형성하는 함수
def initialPopulation(popSize,cityList):
    population = []
    for i in range(0,popSize):
        population.append(createRoute(cityList))
    return population

Determine Fitness
Fitness 이용해 rank만들기

In [6]:
def rankRoutes(population):
    fitnessResults={}
    for i in range(0,len(population)):
        fitnessResults[i] = Fitness(population[i]).routeFitness()
    return sorted(fitnessResults.items(),key = operator.itemgetter(1),reverse=True)

In [7]:
def selection(popRanked,eliteSize):
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked),columns = ['index','Fitness'])
    df['cum_sum']= df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
    
    for i in range(0,eliteSize):
        selectionResults.append(popRanked[i][0])
    for i in range(0,len(popRanked)-eliteSize):
        pick = 100*random.random()
        for i in range(0,len(popRanked)):
            if pick<= df.iat[i,3]:
                selectionResults.append(popRanked[i][0])
                break
    return selectionResults

In [8]:
def matingPool(population,selectionResults):
    matingpool = []
    for i in range(0,len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool

Breed

ordered crossover 사용

In [9]:
def breed(parent1,parent2):
    child = []
    childP1 = []
    childP2 = []
    
    geneA = int(random.random()*len(parent1))
    geneB = int(random.random()*len(parent1))
    startGene = min(geneA,geneB)
    endGene = max(geneA,geneB)
    
    for i in range(startGene,endGene):
        childP1.append(parent1[i])
        
    childP2 = [item for item in parent2 if item not in childP1]
    child = childP1+childP2
    return child
    

In [10]:
def breedPopulation(matingpool,eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool,len(matingpool))
    for i in range(0,eliteSize):
        children.append(matingpool[i])
        
    for i in range(0,length):
        child = breed(pool[i],pool[len(matingpool)-i-1])
        children.append(child)
    return children

Mutate

In [11]:
#swamp mutate
def mutate(individual,mutationRate):
    for swapped in range(len(individual)):
        if(random.random()<mutationRate):
            swapWith = int(random.random()*len(individual))
            
            city1 = individual[swapped]
            city2 = individual[swapWith]
            
            individual[swapped] = city2
            individual[swapWith] = city1
        return individual

In [12]:
def mutatePopulation(population,mutationRate):
    mutatedPop = []
    for ind in range(0,len(population)):
        mutatedInd = mutate(population[ind],mutationRate)
        mutatedPop.append(mutatedInd)
    return mutatedPop

In [13]:
##Repeat - 반복작업!
def nextGeneration(currentGen,eliteSize,mutationRate):
    popRanked = rankRoutes(currentGen)
    selectionResults = selection(popRanked,eliteSize)
    matingpool = matingPool(currentGen,selectionResults)
    children = breedPopulation(matingpool,eliteSize)
    nextGeneration = mutatePopulation(children,mutationRate)
    return nextGeneration

In [24]:
def GeneticAlgorithm(population,popSize,eliteSize,mutationRate,generations):
    start = time.time()
    pop = initialPopulation(popSize,population)
    print('intital distance:'+str(1/rankRoutes(pop)[0][1]))
    progress=[]
    bestroute = 1/rankRoutes(pop)[0][0]
    Finaldistance = 1/rankRoutes(pop)[0][1]
    progress.append(Finaldistance)
    for i in range(0,generations):
        pop = nextGeneration(pop,eliteSize,mutationRate)
        progress.append(1/rankRoutes(pop)[0][1])
        if Finaldistance > 1/rankRoutes(pop)[0][1]:
            bestroute = pop[rankRoutes(pop)[0][0]]
            Finaldistance = 1/rankRoutes(pop)[0][1]
    
    finish = time.time()
    print('Final distance:'+str(Finaldistance))
    print('Time:'+str(finish-start))
    plt.plot(progress)
    plt.xlabel('Generation')
    plt.ylabel('Distance')
    plt.show
    
    
    return bestroute

In [25]:
cityList = []
for i in range(0,35) :
    cityList.append(City(x=int(random.random()*100),y=int(random.random()*100)))

In [26]:
cityList

[(31,73),
 (74,47),
 (54,99),
 (78,34),
 (6,73),
 (63,15),
 (26,89),
 (59,87),
 (62,67),
 (42,78),
 (76,70),
 (26,81),
 (4,54),
 (92,70),
 (8,62),
 (90,40),
 (92,76),
 (94,89),
 (87,7),
 (59,33),
 (52,91),
 (27,30),
 (66,44),
 (70,59),
 (4,34),
 (40,77),
 (15,95),
 (9,46),
 (96,48),
 (67,73),
 (45,88),
 (79,29),
 (17,29),
 (53,43),
 (0,2)]

In [27]:
GeneticAlgorithm(population=cityList,popSize=50,eliteSize=10,mutationRate=0.1,generations=1000)

intital distance:1527.890446859265
Final distance:595.0067890710408


[(66,44),
 (59,33),
 (63,15),
 (87,7),
 (79,29),
 (78,34),
 (90,40),
 (96,48),
 (74,47),
 (53,43),
 (17,29),
 (27,30),
 (0,2),
 (4,34),
 (8,62),
 (4,54),
 (9,46),
 (6,73),
 (15,95),
 (26,89),
 (26,81),
 (31,73),
 (42,78),
 (40,77),
 (54,99),
 (52,91),
 (45,88),
 (59,87),
 (76,70),
 (92,70),
 (92,76),
 (94,89),
 (67,73),
 (62,67),
 (70,59)]